# LGMB with random split for early stopping

해당 노브툭은 https://www.kaggle.com/code/mlisovyi/feature-engineering-lighgbm-with-f1-macro/notebook 을 fork 하여 다음 사항들을 개선하였다.
* LightGBM 모델이 XGBoost로 교체되었으며 그에 따라 코드가 업데이트되었습니다.
* 또한 랜덤포레스트의 투표분류기를 장착하고 XGB의 결과를 RF와 조합하고 있습니다.
* 몇 가지 추가 기능이 추가되었습니다.
* 이전에 삭제되었던 일부 기능은 그대로 유지되었습니다.
* 일부 코드가 재구성되었습니다.
* 데이터를 한 번 분할하여 LGBM 조기 정지를 위한 검증 데이터로 사용하는 대신, 훈련 중에 데이터를 분할하여 전체 훈련 집합을 훈련할 수 있도록 했습니다. 이 경우 K-배 분할보다 이 방법이 더 효과적이라는 것을 알게 되었습니다.

일부 추가 기능은 https://www.kaggle.com/code/kuriyaman1002/reduce-features-140-85-keeping-f1-score?scriptVersionId=4854294 노트북에서 가져왔다.

## Notes from Original Kernel (edited by EAS):

해당 커널은 https://www.kaggle.com/mlisovyi/lighgbm-hyperoptimisation-with-f1-macro 을 따르지만 하이퍼파라미터 최적화 대신 커널 최적값을 사용해 더 빠르게 실행된다.

몇 가지 핵심 사항:

* 이 커널은 가구에 대한 집계를 추출한 후 가구주에 대해서만 훈련을 실행합니다. 이는 공지된 채점 시작 방법을 따릅니다: 채점에는 세대주만 사용된다는 점에 유의하세요. 모든 가구 구성원은 시험 + 샘플 제출에 포함되지만 가구주만 채점됩니다. (데이터 설명에서). 그러나 현재로서는 세대주가 아닌 가구원도 평가에 포함되는 것으로 보입니다(https://www.kaggle.com/c/costa-rican-household-poverty-prediction/discussion/61403#360115 참조). 실제로, 완전 예측은 ~ 0.4 PLB 점수를 제공하는 반면, 모든 비 세대주 항목을 1 등급으로 대체하면 ~ 0.2 PLB 점수로 떨어집니다.
* 클래스 주파수의 균형을 맞추는 것이 매우 중요한 것 같습니다. 밸런싱을 하지 않고 훈련된 모델은 ~0.39 PLB / ~0.43 로컬 테스트를 제공하는 반면, 밸런싱을 추가하면 ~0.42 PLB / 0.47 로컬 테스트를 제공합니다. 밸런싱은 수작업으로 할 수도 있고 언더샘플링을 통해 달성할 수도 있습니다. 그러나 가장 간단하고 언더샘플링에 비해 더 강력한 방법은 sklearn API의 LightGBM 모델 생성자에서 class_weight='balanced'를 설정하는 것입니다.
* 이 커널은 매크로 F1 점수를 사용하여 훈련에서 조기 중단합니다. 이는 채점 전략에 맞추기 위해 수행됩니다.
* 범주형은 블라인드 레이블 인코딩 대신 적절한 매핑을 통해 숫자로 변환됩니다.
* 레이블 인코딩으로 바꾸면 트리 모델에서 더 쉽게 소화할 수 있으므로 OHE를 사용합니다. 이 트릭은 트리가 아닌 모델에는 해로울 수 있으므로 주의하세요.
* idhogar는 훈련에 사용되지 않습니다. 정보가 유출될 수 있는 유일한 방법은 데이터 유출이 있을 때뿐입니다. 우리는 여기서 빈곤과 싸우고 있습니다. 유출을 악용한다고 해서 빈곤이 줄어들지는 않습니다.)
* 가구 내에서 집계가 이루어지며 새로운 기능은 수작업으로 만들어집니다. 대부분의 기능이 이미 가구 단위로 집계되어 있기 때문에 집계할 수 있는 기능이 많지 않다는 점에 유의하세요.
* 투표 분류기는 여러 LightGBM 모델에 대한 평균을 내기 위해 사용됩니다.

In [1]:
import numpy as np # linear algebra
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline

import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import f1_score
# from sklearn.externals.joblib import Parallel, delayed
from joblib import Parallel, delayed
from sklearn.base import clone
from sklearn.ensemble import VotingClassifier, ExtraTreesClassifier, RandomForestClassifier
from sklearn.utils import class_weight

import warnings
warnings.filterwarnings("ignore")

In [2]:
from sklearn.preprocessing import LabelEncoder

# this only transforms the idhogar field, the other things this function used to do are done elsewhere
def encode_data(df):
    df['idhogar'] = LabelEncoder().fit_transform(df['idhogar'])

# plot feature importance for sklearn decision trees    
def feature_importance(forest, X_train, display_results=True):
    ranked_list = []
    zero_features = []
    
    importances = forest.feature_importances_

    indices = np.argsort(importances)[::-1]
    
    if display_results:
        # Print the feature ranking
        print("Feature ranking:")

    for f in range(X_train.shape[1]):
        if display_results:
            print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]) + " - " + X_train.columns[indices[f]])
        
        ranked_list.append(X_train.columns[indices[f]])
        
        if importances[indices[f]] == 0.0:
            zero_features.append(X_train.columns[indices[f]])
            
    return ranked_list, zero_features

In [3]:
def do_features(df):
    feats_div = [('children_fraction', 'r4t1', 'r4t3'), 
                 ('working_man_fraction', 'r4h2', 'r4t3'),
                 ('all_man_fraction', 'r4h3', 'r4t3'),
                 ('human_density', 'tamviv', 'rooms'),
                 ('human_bed_density', 'tamviv', 'bedrooms'),
                 ('rent_per_person', 'v2a1', 'r4t3'),
                 ('rent_per_room', 'v2a1', 'rooms'),
                 ('mobile_density', 'qmobilephone', 'r4t3'),
                 ('tablet_density', 'v18q1', 'r4t3'),
                 ('mobile_adult_density', 'qmobilephone', 'r4t2'),
                 ('tablet_adult_density', 'v18q1', 'r4t2'),
                ]
    
    feats_sub = [('people_not_living', 'tamhog', 'tamviv'),
                 ('people_weird_stat', 'tamhog', 'r4t3')]

    for f_new, f1, f2 in feats_div:
        df['fe_' + f_new] = (df[f1] / df[f2]).astype(np.float32)       
    for f_new, f1, f2 in feats_sub:
        df['fe_' + f_new] = (df[f1] - df[f2]).astype(np.float32)
    
    # aggregation rules over household
    aggs_num = {'age': ['min', 'max', 'mean'],
                'escolari': ['min', 'max', 'mean']
               }
    
    aggs_cat = {'dis': ['mean']}
    for s_ in ['estadocivil', 'parentesco', 'instlevel']:
        for f_ in [f_ for f_ in df.columns if f_.startswith(s_)]:
            aggs_cat[f_] = ['mean', 'count']

    # aggregation over household
    for name_, df_ in [('18', df.query('age >= 18'))]:
        df_agg = df_.groupby('idhogar').agg({**aggs_num, **aggs_cat}).astype(np.float32)
        df_agg.columns = pd.Index(['agg' + name_ + '_' + e[0] + "_" + e[1].upper() for e in df_agg.columns.tolist()])
        df = df.join(df_agg, how='left', on='idhogar')
        del df_agg

    # Drop id's
    df.drop(['Id'], axis=1, inplace=True)
    
    return df

In [4]:
# convert one hot encoded fields to label encoding
def convert_OHE2LE(df):
    tmp_df = df.copy(deep=True)
    for s_ in ['pared', 'piso', 'techo', 'abastagua', 'sanitario', 'energcocinar', 'elimbasu', 
               'epared', 'etecho', 'eviv', 'estadocivil', 'parentesco', 
               'instlevel', 'lugar', 'tipovivi',
               'manual_elec']:
        if 'manual_' not in s_:
            cols_s_ = [f_ for f_ in df.columns if f_.startswith(s_)]
        elif 'elec' in s_:
            cols_s_ = ['public', 'planpri', 'noelec', 'coopele']
        sum_ohe = tmp_df[cols_s_].sum(axis=1).unique()
        #deal with those OHE, where there is a sum over columns == 0
        if 0 in sum_ohe:
            print('The OHE in {} is incomplete. A new column will be added before label encoding'
                  .format(s_))
            # dummy colmn name to be added
            col_dummy = s_+'_dummy'
            # add the column to the dataframe
            tmp_df[col_dummy] = (tmp_df[cols_s_].sum(axis=1) == 0).astype(np.int8)
            # add the name to the list of columns to be label-encoded
            cols_s_.append(col_dummy)
            # proof-check, that now the category is complete
            sum_ohe = tmp_df[cols_s_].sum(axis=1).unique()
            if 0 in sum_ohe:
                 print("The category completion did not work")
        tmp_cat = tmp_df[cols_s_].idxmax(axis=1)
        tmp_df[s_ + '_LE'] = LabelEncoder().fit_transform(tmp_cat).astype(np.int16)
        if 'parentesco1' in cols_s_:
            cols_s_.remove('parentesco1')
        tmp_df.drop(cols_s_, axis=1, inplace=True)
    return tmp_df

# Read in the data and clean it up

In [5]:
train = pd.read_csv('../input/Costa Rican Household Poverty Level Prediction/train.csv')
test = pd.read_csv('../input/Costa Rican Household Poverty Level Prediction/test.csv')

test_ids = test.Id

In [6]:
def process_df(df_):
    # encode the idhogar
    encode_data(df_)
    
    # create aggregate features
    return do_features(df_)

train = process_df(train)
test = process_df(test)

일부 누락된 데이터를 정리하고 개체를 숫자로 변환합니다.

In [7]:
# some dependencies are Na, fill those with the square root of the square
train['dependency'] = np.sqrt(train['SQBdependency'])
test['dependency'] = np.sqrt(test['SQBdependency'])

# fill "no"s for education with 0s
train.loc[train['edjefa'] == "no", 'edjefa'] = 0
train.loc[train['edjefe'] == "no", 'edjefe'] = 0
test.loc[test['edjefa'] == "no", 'edjefa'] = 0
test.loc[test['edjefe'] == "no", 'edjefe'] = 0

# if education is "yes" and person is head of household, fill with escolari
train.loc[(train['edjefa'] == "yes") & (train['parentesco1'] == 1), 'edhefa'] = train.loc[(train['edjefa'] == "yes") & (train['parentesco1'] == 1), 'escolari']
train.loc[(train['edjefe'] == "yes") & (train['parentesco1'] == 1), 'edhefe'] = train.loc[(train['edjefe'] == "yes") & (train['parentesco1'] == 1), 'escolari']

test.loc[(test['edjefa'] == "yes") & (test['parentesco1'] == 1), 'edhefa'] = test.loc[(train['edjefa'] == "yes") & (test['parentesco1'] == 1), 'escolari']
test.loc[(test['edjefe'] == "yes") & (test['parentesco1'] == 1), 'edhefe'] = test.loc[(train['edjefe'] == "yes") & (test['parentesco1'] == 1), 'escolari']

# this field is supposed th be interaction between gender and escolari, but it isn't clear what "yes" means, let's fill it with 4
train.loc[train['edjefa'] == "yes", 'edjefa'] = 4
train.loc[train['edjefe'] == "yes", 'edjefe'] = 4

test.loc[test['edjefa'] == "yes", 'edjefa'] = 4
test.loc[test['edjefe'] == "yes", 'edjefe'] = 4

# convert to int for our models
train['edjefa'] = train['edjefa'].astype("int")
train['edjefe']=  train['edjefe'].astype("int")
test['edjefa'] = test['edjefa'].astype("int")
test['edjefe']=  test['edjefe'].astype("int")

# create feature with max education of either head of household
train['edjef'] = np.max(train[['edjefa', 'edjefe']], axis=1)
test['edjef'] = np.max(train[['edjefa', 'edjefe']], axis=1)

# fill some nas
train['v2a1'] = train['v2a1'].fillna(0)
test['v2a1'] = test['v2a1'].fillna(0)

train['v18q1'] = train['v18q1'].fillna(0)
test['v18q1'] = train['v18q1'].fillna(0)

train['rez_esc'] = train['rez_esc'].fillna(0)
test['rez_esc'] = test['rez_esc'].fillna(0)

train.loc[train.meaneduc.isnull(), "meaneduc"] = 0
train.loc[train.SQBmeaned.isnull(), "SQBmeaned"] = 0

test.loc[test.meaneduc.isnull(), "meaneduc"] = 0
test.loc[test.SQBmeaned.isnull(), "SQBmeaned"] = 0

# fix some inconsistencies in the data - some rows indicate both the household does and does not have a totilet.
# it the is no water we'll assume they do not
train.loc[(train.v14a == 1) & (train.sanitario1 == 1) & (train.abastaguano == 0), "v14a"] = 0
train.loc[(train.v14a == 1) & (train.sanitario1 == 1) & (train.abastaguano == 0), "sanitario1" ] = 0

test.loc[(test.v14a == 1) & (test.sanitario1 == 1) & (test.abastaguano == 0), "v14a"] = 0
test.loc[(test.v14a == 1) & (test.sanitario1 == 1) & (test.abastaguano == 0), "sanitario1" ] = 0

In [8]:
def train_test_apply_func(train_, test_, func_):
    test_['Target'] = 0
    xx = pd.concat([train_, test_])

    xx_func = func_(xx)
    train_ = xx_func.iloc[:train_.shape[0], :]
    test_  = xx_func.iloc[train_.shape[0]:, :].drop('Target', axis=1)

    del xx, xx_func
    return train_, test_

In [9]:
# convert the one hot field into label encoded
train, test = train_test_apply_func(train, test, convert_OHE2LE)

The OHE in techo is incomplete. A new column will be added before label encoding
The OHE in instlevel is incomplete. A new column will be added before label encoding
The OHE in manual_elec is incomplete. A new column will be added before label encoding


# Geo aggregates

In [10]:
cols_2_ohe = ['eviv_LE', 'etecho_LE', 'epared_LE', 'elimbasu_LE', 
              'energcocinar_LE', 'sanitario_LE', 'manual_elec_LE',
              'pared_LE']
cols_nums = ['age', 'meaneduc', 'dependency', 
             'hogar_nin', 'hogar_adul', 'hogar_mayor', 'hogar_total',
             'bedrooms', 'overcrowding']

def convert_geo2aggs(df_):
    tmp_df = pd.concat([df_[(['lugar_LE', 'idhogar']+cols_nums)],
                        pd.get_dummies(df_[cols_2_ohe], 
                                       columns=cols_2_ohe)],axis=1)

    geo_agg = tmp_df.groupby(['lugar_LE','idhogar']).mean().groupby('lugar_LE').mean().astype(np.float32)
    geo_agg.columns = pd.Index(['geo_' + e for e in geo_agg.columns.tolist()])
    
    del tmp_df
    return df_.join(geo_agg, how='left', on='lugar_LE')

# add some aggregates by geography
train, test = train_test_apply_func(train, test, convert_geo2aggs)

In [11]:
# add the number of people over 18 in each household
train['num_over_18'] = 0
train['num_over_18'] = train[train.age >= 18].groupby('idhogar').idhogar.transform("count")
train['num_over_18'] = train.groupby("idhogar")["num_over_18"].transform("max")
train['num_over_18'] = train['num_over_18'].fillna(0)

test['num_over_18'] = 0
test['num_over_18'] = test[test.age >= 18].groupby('idhogar').idhogar.transform("count")
test['num_over_18'] = test.groupby("idhogar")["num_over_18"].transform("max")
test['num_over_18'] = test['num_over_18'].fillna(0)

# add some extra features, these were taken from another kernel
def extract_features(df):
    df['bedrooms_to_rooms'] = df['bedrooms']/df['rooms']
    df['rent_to_rooms'] = df['v2a1']/df['rooms']
    df['tamhog_to_rooms'] = df['tamhog']/df['rooms'] # tamhog - size of the household
    df['r4t3_to_tamhog'] = df['r4t3']/df['tamhog'] # r4t3 - Total persons in the household
    df['r4t3_to_rooms'] = df['r4t3']/df['rooms'] # r4t3 - Total persons in the household
    df['v2a1_to_r4t3'] = df['v2a1']/df['r4t3'] # rent to people in household
    df['v2a1_to_r4t3'] = df['v2a1']/(df['r4t3'] - df['r4t1']) # rent to people under age 12
    df['hhsize_to_rooms'] = df['hhsize']/df['rooms'] # rooms per person
    df['rent_to_hhsize'] = df['v2a1']/df['hhsize'] # rent to household size
    df['rent_to_over_18'] = df['v2a1']/df['num_over_18']
    # some households have no one over 18, use the total rent for those
    df.loc[df.num_over_18 == 0, "rent_to_over_18"] = df[df.num_over_18 == 0].v2a1
    
extract_features(train)    
extract_features(test)   

In [12]:
# drop duplicated columns
needless_cols = ['r4t3', 'tamhog', 'tamviv', 'hhsize', 'v18q', 'v14a', 'agesq',
                 'mobilephone', 'female', ]

instlevel_cols = [s for s in train.columns.tolist() if 'instlevel' in s]

needless_cols.extend(instlevel_cols)

train = train.drop(needless_cols, axis=1)
test = test.drop(needless_cols, axis=1)

## Split the data

같은 가구에 속한 행은 일반적으로 동일한 타겟을 갖기 때문에 누출을 방지하기 위해 데이터를 가구별로 분할합니다. 세대주만 포함하도록 데이터를 필터링하기 때문에 기술적으로 반드시 필요한 것은 아니지만, 그렇게 하려는 경우 전체 학습 데이터 세트를 쉽게 사용할 수 있는 방법을 제공합니다.

데이터를 분할한 후에는 전체 데이터 세트로 학습 데이터를 덮어쓰므로 모든 데이터에 대해 학습할 수 있습니다. split_data 함수는 데이터를 덮어쓰지 않고 동일한 작업을 수행하며, 훈련 루프 내에서 (희망적으로) K-Fold 분할을 근사화하는 데 사용됩니다.

In [13]:
def split_data(train, y, sample_weight=None, households=None, test_percentage=0.20, seed=None):
    # uncomment for extra randomness
    # np.random.seed(seed=seed)
    
    train2 = train.copy()
    
    # pick some random households to use for the test data
    cv_hhs = np.random.choice(households, size=int(len(households) * test_percentage), replace=False)
    
    # select households which are in the random selection
    cv_idx = np.isin(households, cv_hhs)
    X_test = train2[cv_idx]
    y_test = y[cv_idx]

    X_train = train2[~cv_idx]
    y_train = y[~cv_idx]
    
    if sample_weight is not None:
        y_train_weights = sample_weight[~cv_idx]
        return X_train, y_train, X_test, y_test, y_train_weights
    
    return X_train, y_train, X_test, y_test

In [14]:
X = train.query('parentesco1==1')
# X = train.copy()

# pull out and drop the target variable
y = X['Target'] - 1
X = X.drop(['Target'], axis=1)

np.random.seed(seed=None)

train2 = X.copy()

train_hhs = train2.idhogar

households = train2.idhogar.unique()
cv_hhs = np.random.choice(households, size=int(len(households) * 0.15), replace=False)

cv_idx = np.isin(train2.idhogar, cv_hhs)

X_test = train2[cv_idx]
y_test = y[cv_idx]

X_train = train2[~cv_idx]
y_train = y[~cv_idx]

# train on entire dataset
X_train = train2
y_train = y

train_households = X_train.idhogar

In [15]:
# figure out the class weights for training with unbalanced classes
y_train_weights = class_weight.compute_sample_weight('balanced', y_train, indices=None)

In [16]:
# drop some features which aren't used by the LGBM or have very low importance
extra_drop_features = [
 'agg18_estadocivil1_MEAN',
 'agg18_estadocivil6_COUNT',
 'agg18_estadocivil7_COUNT',
 'agg18_parentesco10_COUNT',
 'agg18_parentesco11_COUNT',
 'agg18_parentesco12_COUNT',
 'agg18_parentesco1_COUNT',
 'agg18_parentesco2_COUNT',
 'agg18_parentesco3_COUNT',
 'agg18_parentesco4_COUNT',
 'agg18_parentesco5_COUNT',
 'agg18_parentesco6_COUNT',
 'agg18_parentesco7_COUNT',
 'agg18_parentesco8_COUNT',
 'agg18_parentesco9_COUNT',
 'geo_elimbasu_LE_4',
 'geo_energcocinar_LE_1',
 'geo_energcocinar_LE_2',
 'geo_epared_LE_0',
 'geo_hogar_mayor',
 'geo_manual_elec_LE_2',
 'geo_pared_LE_3',
 'geo_pared_LE_4',
 'geo_pared_LE_5',
 'geo_pared_LE_6',
 'num_over_18',
 'parentesco_LE',
 'rez_esc']

In [17]:
xgb_drop_cols = extra_drop_features + ["idhogar",  'parentesco1']

# Fit a voting classifier

조기 중지를 위해 fit_params를 전달할 수 있도록 파생된 투표 분류기 클래스를 정의합니다. 매크로 F1과 감쇠 학습률을 기반으로 조기 중지를 사용하는 LGBM 모델을 기반으로 투표합니다.

파라미터는 이 커널에서 무작위 검색을 통해 최적화됩니다: https://www.kaggle.com/mlisovyi/lighgbm-hyperoptimisation-with-f1-macro

In [18]:
# 4
opt_parameters = {'max_depth':35, 'eta':0.1, 'verbosity':0, 'objective':'multi:softmax', 'min_child_weight': 1, 'num_class': 4, 'gamma': 2.0, 'colsample_bylevel': 0.9, 'subsample': 0.84, 'colsample_bytree': 0.88, 'reg_lambda': 0.40 }
# 5
opt_parameters = {'max_depth':35, 'eta':0.15, 'verbosity':1, 'objective':'multi:softmax', 'min_child_weight': 2, 'num_class': 4, 'gamma': 2.5, 'colsample_bylevel': 1, 'subsample': 0.95, 'colsample_bytree': 0.85, 'reg_lambda': 0.35 }
# 6
# opt_parameters = {'max_depth':35, 'eta':0.15, 'verbosity':0, 'objective':'multi:softmax', 'min_child_weight': 2, 'num_class': 4, 'gamma': 2.75, 'colsample_bylevel': 0.95, 'subsample': 0.95, 'colsample_bytree': 0.85, 'reg_lambda': 0.35 }
# # 7
# opt_parameters = {'max_depth':35, 'eta':0.12, 'verbosity':0, 'objective':'multi:softmax', 'min_child_weight': 2, 'num_class': 4, 'gamma': 3.25, 'colsample_bylevel': 0.95, 'subsample': 0.88, 'colsample_bytree': 0.88, 'reg_lambda': 0.35 }

def evaluate_macroF1_lgb(predictions, truth):  
    # this follows the discussion in https://github.com/Microsoft/LightGBM/issues/1483
    pred_labels = predictions.argmax()
    truth = truth.get_label()
    
    f1 = f1_score(truth, predictions, average='macro')
    return ('macroF1', 1-f1) 

fit_params={"early_stopping_rounds":500,
            "eval_metric" : evaluate_macroF1_lgb, 
            "eval_set" : [(X_train,y_train), (X_test,y_test)],
            'verbose': False,
           }

def learning_rate_power_0997(current_iter):
    base_learning_rate = 0.1
    min_learning_rate = 0.02
    lr = base_learning_rate  * np.power(.995, current_iter)
    return max(lr, min_learning_rate)

fit_params['verbose'] = 50

In [19]:
np.random.seed(100)

def _parallel_fit_estimator(estimator1, X, y, sample_weight=None, threshold=True, **fit_params):
    estimator = clone(estimator1)
    
    # randomly split the data so we have a test set for early stopping
    if sample_weight is not None:
        X_train, y_train, X_test, y_test, y_train_weight = split_data(X, y, sample_weight, households=train_households)
    else:
        X_train, y_train, X_test, y_test = split_data(X, y, None, households=train_households)
        
    # update the fit params with our new split
    fit_params["eval_set"] = [(X_test,y_test)]
    
    # fit the estimator
    if sample_weight is not None:
        if isinstance(estimator1, ExtraTreesClassifier) or isinstance(estimator1, RandomForestClassifier):
            estimator.fit(X_train, y_train)
        else:
            _ = estimator.fit(X_train, y_train, sample_weight=y_train_weight, **fit_params)
    else:
        if isinstance(estimator1, ExtraTreesClassifier) or isinstance(estimator1, RandomForestClassifier):
            estimator.fit(X_train, y_train)
        else:
            _ = estimator.fit(X_train, y_train, **fit_params)
    
    if not isinstance(estimator1, ExtraTreesClassifier) and not isinstance(estimator1, RandomForestClassifier) and not isinstance(estimator1, xgb.XGBClassifier):
        best_cv_round = np.argmax(estimator.evals_result_['validation_0']['mlogloss'])
        best_cv = np.max(estimator.evals_result_['validation_0']['mlogloss'])
        best_train = estimator.evals_result_['train']['macroF1'][best_cv_round]
    else:
        best_train = f1_score(y_train, estimator.predict(X_train), average="macro")
        best_cv = f1_score(y_test, estimator.predict(X_test), average="macro")
        print("Train F1:", best_train)
        print("Test F1:", best_cv)
        
    # reject some estimators based on their performance on train and test sets
    if threshold:
        # if the valid score is very high we'll allow a little more leeway with the train scores
        if ((best_cv > 0.37) and (best_train > 0.75)) or ((best_cv > 0.44) and (best_train > 0.65)):
            return estimator

        # else recurse until we get a better one
        else:
            print("Unacceptable!!! Trying again...")
            return _parallel_fit_estimator(estimator1, X, y, sample_weight=sample_weight, **fit_params)
    
    else:
        return estimator
    
class VotingClassifierLGBM(VotingClassifier):
    '''
    This implements the fit method of the VotingClassifier propagating fit_params
    '''
    def fit(self, X, y, sample_weight=None, threshold=True, **fit_params):
        
        if isinstance(y, np.ndarray) and len(y.shape) > 1 and y.shape[1] > 1:
            raise NotImplementedError('Multilabel and multi-output'
                                      ' classification is not supported.')

        if self.voting not in ('soft', 'hard'):
            raise ValueError("Voting must be 'soft' or 'hard'; got (voting=%r)"
                             % self.voting)

        if self.estimators is None or len(self.estimators) == 0:
            raise AttributeError('Invalid `estimators` attribute, `estimators`'
                                 ' should be a list of (string, estimator)'
                                 ' tuples')

        if (self.weights is not None and
                len(self.weights) != len(self.estimators)):
            raise ValueError('Number of classifiers and weights must be equal'
                             '; got %d weights, %d estimators'
                             % (len(self.weights), len(self.estimators)))

        names, clfs = zip(*self.estimators)
        self._validate_names(names)

        n_isnone = np.sum([clf is None for _, clf in self.estimators])
        if n_isnone == len(self.estimators):
            raise ValueError('All estimators are None. At least one is '
                             'required to be a classifier!')

        self.le_ = LabelEncoder().fit(y)
        self.classes_ = self.le_.classes_
        self.estimators_ = []

        transformed_y = self.le_.transform(y)

        self.estimators_ = Parallel(n_jobs=self.n_jobs)(
                delayed(_parallel_fit_estimator)(clone(clf), X, transformed_y,
                                                 sample_weight=sample_weight, threshold=threshold, **fit_params)
                for clf in clfs if clf is not None)

        return self

In [20]:
clfs = []
for i in range(15):
    clf = xgb.XGBClassifier(random_state=217+i, n_estimators=300, learning_rate=0.15, n_jobs=4, **opt_parameters)
    
    clfs.append(('xgb{}'.format(i), clf))
    
vc = VotingClassifierLGBM(clfs, voting='soft')
del(clfs)

#Train the final model with learning rate decay

_ = vc.fit(X_train.drop(xgb_drop_cols, axis=1), y_train, sample_weight=y_train_weights, threshold=False, **fit_params)

clf_final = vc.estimators_[0]

[0]	validation_0-mlogloss:1.30152	validation_0-macroF1:0.65445
[50]	validation_0-mlogloss:0.91929	validation_0-macroF1:0.56136
[100]	validation_0-mlogloss:0.91704	validation_0-macroF1:0.56791
[150]	validation_0-mlogloss:0.91598	validation_0-macroF1:0.56757
[200]	validation_0-mlogloss:0.91414	validation_0-macroF1:0.56898
[250]	validation_0-mlogloss:0.91364	validation_0-macroF1:0.57355
[299]	validation_0-mlogloss:0.91400	validation_0-macroF1:0.57779
Train F1: 0.8935168893176608
Test F1: 0.44551431974757316
[0]	validation_0-mlogloss:1.31088	validation_0-macroF1:0.65553
[50]	validation_0-mlogloss:0.91842	validation_0-macroF1:0.56830
[100]	validation_0-mlogloss:0.91848	validation_0-macroF1:0.59189
[150]	validation_0-mlogloss:0.91699	validation_0-macroF1:0.57406
[200]	validation_0-mlogloss:0.91389	validation_0-macroF1:0.58228
[250]	validation_0-mlogloss:0.91314	validation_0-macroF1:0.58592
[299]	validation_0-mlogloss:0.91198	validation_0-macroF1:0.58296
Train F1: 0.8977184399356206
Test F1: 

In [21]:
# params 4 - 400 early stop - 15 estimators - l1 used features - weighted
global_score = f1_score(y_test, clf_final.predict(X_test.drop(xgb_drop_cols, axis=1)), average='macro')
vc.voting = 'soft'
global_score_soft = f1_score(y_test, vc.predict(X_test.drop(xgb_drop_cols, axis=1)), average='macro')
vc.voting = 'hard'
global_score_hard = f1_score(y_test, vc.predict(X_test.drop(xgb_drop_cols, axis=1)), average='macro')

print('Validation score of a single LGBM Classifier: {:.4f}'.format(global_score))
print('Validation score of a VotingClassifier on 3 LGBMs with soft voting strategy: {:.4f}'.format(global_score_soft))
print('Validation score of a VotingClassifier on 3 LGBMs with hard voting strategy: {:.4f}'.format(global_score_hard))

Validation score of a single LGBM Classifier: 0.8398
Validation score of a VotingClassifier on 3 LGBMs with soft voting strategy: 0.9208
Validation score of a VotingClassifier on 3 LGBMs with hard voting strategy: 0.9156


In [22]:
# see which features are not used by ANY models
useless_features = []
drop_features = set()
counter = 0
for est in vc.estimators_:
    ranked_features, unused_features = feature_importance(est, X_train.drop(xgb_drop_cols, axis=1), display_results=False)
    useless_features.append(unused_features)
    if counter == 0:
        drop_features = set(unused_features)
    else:
        drop_features = drop_features.intersection(set(unused_features))
    counter += 1
    
drop_features

{'agg18_estadocivil5_COUNT', 'geo_energcocinar_LE_0', 'geo_manual_elec_LE_3'}

In [23]:
ranked_features = feature_importance(clf_final, X_train.drop(xgb_drop_cols, axis=1))

Feature ranking:
1. feature 59 (0.021402) - agg18_escolari_MAX
2. feature 42 (0.018929) - fe_children_fraction
3. feature 133 (0.018828) - geo_manual_elec_LE_4
4. feature 40 (0.015555) - SQBdependency
5. feature 74 (0.015502) - agg18_parentesco2_MEAN
6. feature 86 (0.012092) - edhefe
7. feature 37 (0.011678) - SQBedjefe
8. feature 60 (0.011528) - agg18_escolari_MEAN
9. feature 126 (0.011523) - geo_sanitario_LE_1
10. feature 22 (0.011509) - dependency
11. feature 122 (0.010849) - geo_elimbasu_LE_5
12. feature 96 (0.010796) - etecho_LE
13. feature 108 (0.010728) - geo_bedrooms
14. feature 103 (0.010532) - geo_meaneduc
15. feature 13 (0.010358) - r4t2
16. feature 12 (0.010064) - r4t1
17. feature 17 (0.010061) - male
18. feature 89 (0.009666) - piso_LE
19. feature 35 (0.009661) - SQBage
20. feature 15 (0.009604) - cielorazo
21. feature 97 (0.009582) - eviv_LE
22. feature 19 (0.009575) - hogar_adul
23. feature 1 (0.009529) - hacdor
24. feature 116 (0.009529) - geo_epared_LE_1
25. feature 95

## Random Forest

In [24]:
et_drop_cols = ['agg18_age_MAX', 'agg18_age_MEAN', 'agg18_age_MIN', 'agg18_dis_MEAN',
       'agg18_escolari_MAX', 'agg18_escolari_MEAN', 'agg18_escolari_MIN',
       'agg18_estadocivil1_COUNT', 'agg18_estadocivil1_MEAN',
       'agg18_estadocivil2_COUNT', 'agg18_estadocivil2_MEAN',
       'agg18_estadocivil3_COUNT', 'agg18_estadocivil3_MEAN',
       'agg18_estadocivil4_COUNT', 'agg18_estadocivil4_MEAN',
       'agg18_estadocivil5_COUNT', 'agg18_estadocivil5_MEAN',
       'agg18_estadocivil6_COUNT', 'agg18_estadocivil6_MEAN',
       'agg18_estadocivil7_COUNT', 'agg18_estadocivil7_MEAN',
       'agg18_parentesco10_COUNT', 'agg18_parentesco10_MEAN',
       'agg18_parentesco11_COUNT', 'agg18_parentesco11_MEAN',
       'agg18_parentesco12_COUNT', 'agg18_parentesco12_MEAN',
       'agg18_parentesco1_COUNT', 'agg18_parentesco1_MEAN',
       'agg18_parentesco2_COUNT', 'agg18_parentesco2_MEAN',
       'agg18_parentesco3_COUNT', 'agg18_parentesco3_MEAN',
       'agg18_parentesco4_COUNT', 'agg18_parentesco4_MEAN',
       'agg18_parentesco5_COUNT', 'agg18_parentesco5_MEAN',
       'agg18_parentesco6_COUNT', 'agg18_parentesco6_MEAN',
       'agg18_parentesco7_COUNT', 'agg18_parentesco7_MEAN',
       'agg18_parentesco8_COUNT', 'agg18_parentesco8_MEAN',
       'agg18_parentesco9_COUNT', 'agg18_parentesco9_MEAN'] #+ ['parentesco_LE', 'rez_esc']

et_drop_cols.extend(["idhogar", "parentesco1", 'fe_rent_per_person', 'fe_rent_per_room',
       'fe_tablet_adult_density', 'fe_tablet_density'])

In [25]:
# do the same thing for some extra trees classifiers
ets = []    
for i in range(10):
    rf = RandomForestClassifier(max_depth=None, random_state=217+i, n_jobs=4, n_estimators=700, min_impurity_decrease=1e-3, min_samples_leaf=2, verbose=0, class_weight="balanced")
    ets.append(('rf{}'.format(i), rf))   

vc2 = VotingClassifierLGBM(ets, voting='soft')  
_ = vc2.fit(X_train.drop(et_drop_cols, axis=1).fillna(0), y_train, threshold=False)    

Train F1: 0.8994073227039447
Test F1: 0.4077767086983677
Train F1: 0.8981928351735758
Test F1: 0.4040791686810088
Train F1: 0.8917778100487993
Test F1: 0.4004984813375832
Train F1: 0.8904000579200092
Test F1: 0.4221753518260778
Train F1: 0.8944843820452583
Test F1: 0.4526988244247092
Train F1: 0.8933756963570761
Test F1: 0.4491052669031149
Train F1: 0.8905908707876973
Test F1: 0.4143903822720081
Train F1: 0.8924615102721188
Test F1: 0.46511308056118383
Train F1: 0.9065299561598678
Test F1: 0.4370083228479642
Train F1: 0.8936010213926351
Test F1: 0.42396487590364973


In [26]:
# w/ threshold, extra drop cols
vc2.voting = 'soft'
global_rf_score_soft = f1_score(y_test, vc2.predict(X_test.drop(et_drop_cols, axis=1).fillna(0)), average='macro')
vc2.voting = 'hard'
global_rf_score_hard = f1_score(y_test, vc2.predict(X_test.drop(et_drop_cols, axis=1).fillna(0)), average='macro')

print('Validation score of a VotingClassifier on 3 LGBMs with soft voting strategy: {:.4f}'.format(global_rf_score_soft))
print('Validation score of a VotingClassifier on 3 LGBMs with hard voting strategy: {:.4f}'.format(global_rf_score_hard))

Validation score of a VotingClassifier on 3 LGBMs with soft voting strategy: 0.8666
Validation score of a VotingClassifier on 3 LGBMs with hard voting strategy: 0.8905


In [27]:
# w/o threshold, extra drop cols
vc2.voting = 'soft'
global_rf_score_soft = f1_score(y_test, vc2.predict(X_test.drop(et_drop_cols, axis=1).fillna(0)), average='macro')
vc2.voting = 'hard'
global_rf_score_hard = f1_score(y_test, vc2.predict(X_test.drop(et_drop_cols, axis=1).fillna(0)), average='macro')

print('Validation score of a VotingClassifier on 3 LGBMs with soft voting strategy: {:.4f}'.format(global_rf_score_soft))
print('Validation score of a VotingClassifier on 3 LGBMs with hard voting strategy: {:.4f}'.format(global_rf_score_hard))

Validation score of a VotingClassifier on 3 LGBMs with soft voting strategy: 0.8666
Validation score of a VotingClassifier on 3 LGBMs with hard voting strategy: 0.8905


In [28]:
# see which features are not used by ANY models
useless_features = []
drop_features = set()
counter = 0
for est in vc2.estimators_:
    ranked_features, unused_features = feature_importance(est, X_train.drop(et_drop_cols, axis=1), display_results=False)
    useless_features.append(unused_features)
    if counter == 0:
        drop_features = set(unused_features)
    else:
        drop_features = drop_features.intersection(set(unused_features))
    counter += 1
    
drop_features

{'parentesco_LE', 'rez_esc'}

In [29]:
def combine_voters(data, weights=[0.5, 0.5]):
    # do soft voting with both classifiers
    vc.voting="soft"
    vc1_probs = vc.predict_proba(data.drop(xgb_drop_cols, axis=1))
    vc2.voting="soft"
    vc2_probs = vc2.predict_proba(data.drop(et_drop_cols, axis=1))
    
    final_vote = (vc1_probs * weights[0]) + (vc2_probs * weights[1])
    predictions = np.argmax(final_vote, axis=1)
    
    return predictions

In [30]:
combo_preds = combine_voters(X_test.fillna(0), weights=[0.5, 0.5])
global_combo_score_soft = f1_score(y_test, combo_preds, average='macro')
global_combo_score_soft

0.9060685954382525

In [31]:
combo_preds = combine_voters(X_test.fillna(0), weights=[0.4, 0.6])
global_combo_score_soft= f1_score(y_test, combo_preds, average='macro')
global_combo_score_soft

0.905543594278918

In [32]:
combo_preds = combine_voters(X_test.fillna(0), weights=[0.6, 0.4])
global_combo_score_soft = f1_score(y_test, combo_preds, average='macro')
global_combo_score_soft

0.9060685954382525

# Prepare submission

In [33]:
y_subm = pd.DataFrame()
y_subm['Id'] = test_ids

In [34]:
vc.voting = 'soft'
y_subm_lgb = y_subm.copy(deep=True)
y_subm_lgb['Target'] = vc.predict(test.drop(xgb_drop_cols, axis=1).fillna(0)) + 1

vc2.voting = 'soft'
y_subm_rf = y_subm.copy(deep=True)
y_subm_rf['Target'] = vc2.predict(test.drop(et_drop_cols, axis=1).fillna(0)) + 1

y_subm_ens = y_subm.copy(deep=True)
y_subm_ens['Target'] = combine_voters(test.fillna(0)) + 1